In [1]:
#Continuation from the previous file, we're looking to generate new data. 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy.ndimage.interpolation import rotate, shift, zoom

In [2]:
def show_digit(val): #Takes a row from a dataframe and prints out the image
    plt.imshow(val.values.reshape(28, 28), cmap = "gray_r")

In [3]:
traindf = pd.read_csv("sub_train.csv", index_col = "Unnamed: 0")
trainLabeldf = pd.read_csv("sub_train_label.csv", index_col = "Unnamed: 0")
traindf

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
trainLabeldf

,label
0,1
1,0
2,1
3,4
4,0
...,...
41995,0
41996,1
41997,7
41998,6


In [5]:
nine_mask = trainLabeldf["label"] == 9
nine_digits = traindf[nine_mask]
nine_digits

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
def shift_image(digit, dx = None, dy = None, rot = None):
    digit = np.array(digit).reshape((28, 28))
    
    if dx != None and dy != None and rot == None:
        s_digit = shift(digit, [dx, dy], cval = 0, mode = "constant")
        s_digit = s_digit.reshape([-1])
        return s_digit, None
    
    if rot != None and dx == None and dy == None:
        r_digit = rotate(digit, rot, cval = 0, mode = "constant", reshape = False)
        r_digit = r_digit.reshape([-1])
        return r_digit, None
    
    s_digit = shift(digit, [dx, dy], cval = 0, mode = "constant")
    s_digit = s_digit.reshape([-1])
    r_digit = rotate(digit, rot, cval = 0, mode = "constant", reshape = False)
    r_digit = r_digit.reshape([-1])
    
    return s_digit, r_digit

In [17]:
def aug_digit(digit):
    aug_digits = []
    for rot in [-20, 0, 20]:
        for dx in [-5, 5]:
            for dy in [-5, 5]:
                d1, d2 = shift_image(digit, dx, dy, rot)
                if d2 is not None:
                    aug_digits.append(np.array(d1))
                    aug_digits.append(np.array(d2))
                else:
                    aug_digits.append(np.array(d1))
    aug_digits.append(np.array(digit))
    return aug_digits

In [18]:
dfNew = pd.DataFrame(columns = [traindf.columns])
dfNew

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783


In [19]:
#DON'T RUN!

In [21]:
import datetime

print(len(traindf.index))

for i in traindf.index:
    print("i is"  + str(i))
    start = datetime.datetime.now()
    aug_digits = aug_digit(traindf.loc[i])
    for j in aug_digits:
        dfNew.loc[len(dfNew)] = pd.Series(j)
    end = datetime.datetime.now()
    timeD = end-start
    print("Difference is " + str(timeD))

42000
i is0
Difference is 0:00:01.042993
i is1
Difference is 0:00:01.078148
i is2
Difference is 0:00:01.062828
i is3
Difference is 0:00:01.059303
i is4
Difference is 0:00:01.085681
i is5
Difference is 0:00:01.100990
i is6
Difference is 0:00:01.126709
i is7
Difference is 0:00:01.153169
i is8
Difference is 0:00:01.175118
i is9
Difference is 0:00:01.252410
i is10
Difference is 0:00:01.294932
i is11
Difference is 0:00:01.247797
i is12
Difference is 0:00:01.264662
i is13
Difference is 0:00:01.271362
i is14
Difference is 0:00:01.404393
i is15
Difference is 0:00:01.325754
i is16
Difference is 0:00:01.348290
i is17
Difference is 0:00:01.377531
i is18
Difference is 0:00:01.414005
i is19
Difference is 0:00:01.418432
i is20
Difference is 0:00:01.425624
i is21
Difference is 0:00:01.447602
i is22
Difference is 0:00:01.483889
i is23
Difference is 0:00:01.485922
i is24
Difference is 0:00:01.549363
i is25
Difference is 0:00:01.539399
i is26
Difference is 0:00:01.551456
i is27
Difference is 0:00:01.568

Difference is 0:00:07.199602
i is226
Difference is 0:00:06.916787
i is227
Difference is 0:00:07.210128
i is228
Difference is 0:00:07.119243
i is229
Difference is 0:00:07.084115
i is230
Difference is 0:00:07.819619
i is231
Difference is 0:00:06.949632
i is232
Difference is 0:00:06.920298
i is233
Difference is 0:00:07.372920
i is234
Difference is 0:00:06.582586
i is235
Difference is 0:00:06.804975
i is236
Difference is 0:00:07.605746
i is237
Difference is 0:00:06.882379
i is238
Difference is 0:00:06.781907
i is239
Difference is 0:00:06.897538
i is240
Difference is 0:00:06.943478
i is241
Difference is 0:00:06.404704
i is242
Difference is 0:00:06.309832
i is243
Difference is 0:00:06.285639
i is244
Difference is 0:00:06.393230
i is245
Difference is 0:00:06.409745
i is246
Difference is 0:00:06.349059
i is247
Difference is 0:00:06.617712
i is248
Difference is 0:00:06.839012
i is249
Difference is 0:00:06.895428
i is250
Difference is 0:00:06.937291
i is251
Difference is 0:00:06.925108
i is252
D

KeyboardInterrupt: 